# Econometria Aplicada

## Aula 1 - REGRESSÃO LINEAR: SIMPLES E MÚLTIPLA



João Ricardo Costa Filho \\
[joaocostafilho.com](https://)

## A regressão linear

**Motivação**: \\

Qual é a exposição de ações individuais ao portfólio de mercado?

### Pacotes

In [ ]:
install.packages("quantmod", quiet = TRUE)

In [ ]:
library(quantmod)

### Dados

Vamos obter dados referentes às ações ordinárias da Petrobrás:

In [ ]:
getSymbols('PETR3.SA',src='yahoo')

Vamos fazer um gráfico rápido com uma funcionalidade do pacote 'quantmod':

In [ ]:
chartSeries(PETR3.SA)

Vamos obter dados referentes às ações ordinárias da Vale:

In [ ]:
getSymbols('VALE3.SA',src='yahoo')

In [ ]:
chartSeries(VALE3.SA)

Quando tivermos dúvidas sobre uma função, podemos utilizar "?":

In [ ]:
?getSymbols

Para selecionarmos as primeiras 15 linhas de um 'dataframe', podemos utilizar o seguinte comando:

In [ ]:
head( PETR3.SA, 15)

E a estrutura dos dados do 'dataframe' pode ser obtida da seguinte forma:

In [ ]:
str( PETR3.SA ) # estrutura dos dados

Vamos obter dados referentes ao Ibovespa:

In [ ]:
getSymbols("^BVSP",src='yahoo')

Veja as primeiras 10 linhas do objeto 'BVSP':

In [ ]:
head(BVSP)

## Visualização dos dados

Vamos criar um objeto no R apenas com o valor de fechamento da ação da Petrobrás:

In [ ]:
petro = PETR3.SA$PETR3.SA.Close

Vamos fazer um gráfico a ação ao longo do tempo:

In [ ]:
plot( petro )

Podemos salvar o gráfico em pdf:

In [ ]:
pdf('petro.pdf')
plot( petro )
dev.off()

Faça o mesmo com a ação da Vale:

Faça o mesmo com o Ibovespa:

Crie uma variável com o ln do preço de fechamento da ação da Petrobrás com a função "log" (se tiver dúvida sobre ela, utlize o comando '?'):

Crie uma variável com o ln do preço de fechamento da ação da Vale com a função "log"

Crie uma variável com o ln do preço de fechamento da ação da Petrobrás com a função "log"

Vamos calcular a primeira diferença de cada um dos três objetos (as ações e o índice):

In [ ]:
dpetr = diff( lnpetr ) * 100

Podemos fazer um histograma dos retorno diários das ações da Petrobrás:

In [ ]:
hist( dpetr )

Podemos melhorar a estética dos gráficos um pouco mais com o pacote 'ggplot':

In [ ]:
library(ggplot2)

Primeiro, temos que criar um "dataframe" com os nossos dados:

In [ ]:
dat = data.frame( dpetr, dvale, dibov )

Veja as últimas oito linhas do dataframe:

In [ ]:
tail( dat )

Voltando ao nosso gráfico:

In [ ]:
ggplot( dat ) +
  geom_histogram(aes(x = dpetr, y = ..density..), color = "black", fill = "darkred", alpha = 0.8) +
    labs(title = "Histograma dos retornos diários das ações da Petrobrás", x = NULL, y = "Densidade") +
  theme_classic()

Podemos editar o tamanho e a fonte do título e dos eixos:

In [ ]:
 ggplot( dat ) +
  geom_histogram(aes(x = dpetr, y = ..density..), color = "black", fill = "darkred", alpha = 0.8) +
    labs(title = "Histograma dos retornos diários das ações da Petrobrás", x = NULL, y = "Densidade") +
  theme_classic() +
  theme(
    plot.title = element_text(size = 16, face = "bold"),
    axis.title = element_text(size = 14, face = "bold"),
    axis.text = element_text(size = 12),
    strip.text = element_text(size = 12, face = "bold")
  )

Analogamente, faça o histograma das outras variáveis:

Façamos a dispersão entre os retornos diários das ações da Petrobrás e do Ibovespa:

In [ ]:
ggplot( dat, aes( x = dibov, y = dpetr ) ) +
  geom_point( color = 'darkred', size = 3 ) +
  labs(title = "Retornos diários das ações da Petrobrás e do Ibovespa",
       x = "Ibovespa",
       y = "Petrobrás") +
         theme_classic() +
  theme(plot.title = element_text(size = 16, face = "bold"),
        axis.title.x = element_text(size = 14, face = "bold"),
        axis.title.y = element_text(size = 14, face = "bold")
        ) +
  theme(panel.background = element_rect(fill = "#f0f0f0"),
        plot.background = element_rect(fill = "#f0f0f0"),
        panel.grid.major = element_line(colour = "white") )

Façamos também a dispersão entre os retornos diários das ações da Vale e do Ibovespa:

Estatísticas descritivas

In [ ]:
summary( dat )

### Regressão

Queremos estimar a seguinte regressão: \\
\
$r^{\text{petr}}_i = \beta_0 + \beta_1 r^{\text{ibov}}_i + ɛ_i$


In [ ]:
reg = lm( dpetr ~ dibov, data = dat )

summary( reg )

In [ ]:
ggplot( dat, aes( x = dibov, y = dpetr ) ) +
  geom_point( color = 'darkred', size = 3 ) +
  labs(title = "Retornos diários das ações da Petrobrás e do Ibovespa",
       x = "Ibovespa",
       y = "Petrobrás") +
         theme_classic() +
  theme(plot.title = element_text(size = 18, face = "bold"),
        axis.title.x = element_text(size = 14, face = "bold"),
        axis.title.y = element_text(size = 14, face = "bold")
        ) +
  theme(panel.background = element_rect(fill = "#f0f0f0"),
        plot.background = element_rect(fill = "#f0f0f0"),
        panel.grid.major = element_line(colour = "white") )+
     geom_smooth(method = lm, se = FALSE, fullrange = TRUE)


Qual é o valor do retorno diário **esperado** da ação da Petrobrás quando o Ibovespa sobre 2%? E quando cai 1%?

Faça a mesma análise para as ações da Vale.

Queremos estimar a seguinte regressão: \\
\
$r^{\text{vale}}_i = \beta_0 + \beta_1 r^{\text{ibov}}_i + ɛ_i$


Qual é o valor do retorno diário **esperado** da ação da Vale quando o Ibovespa sobre 2%? E quando cai 1%?

## Desafio para casa

Estime $r^{\text{petr}}_i = \beta_0 + \beta_1 \left( r^{\text{ibov}}_i - r^{\text{selic}}_i \right) + ɛ_i$.

## Inferência

Como verificar se a associação entre as variáveis é estatísticamente significativa? Realizados testes de hipótese!

Para $\hat{ \beta}_0$:

$\mathcal{H}_0: \beta_0 = 0$ \\
$\mathcal{H}_a: \beta_0 \neq 0$ \\

Para $\hat{ \beta}_1$:

$\mathcal{H}_0: \beta_1 = 0$ \\
$\mathcal{H}_a: \beta_1 \neq 0$ \\

Vamos simular os dados:

In [ ]:
# Para replicarmos as variáveis pseudo aleatórias
set.seed(1301)

# Definindo os parâmetros

amostras <- 500 # número de amostras

n <- 200        # tamanho de cada amostra

b0 <- 2

b1 <- 3

# Criando as amostras

X <- replicate( amostras, rnorm( n, mean = 10, sd = 2 ) )

print( "Variável X: linhas e colunas")

nrow( X ) # numero de linhas
ncol( X ) # número de colunas

e <- replicate( amostras, rnorm( n, mean = 0, sd = 1 ) )

Y <- b0 + b1 * X + e

print( "Variável Y: linhas e colunas")

nrow( Y ) # numero de linhas
ncol( Y ) # número de colunas

In [ ]:
# Fazendo as regressões

regressoes <- lapply( 1:amostras,
                      function(i) lm( Y[ , i ] ~ X[ , i ] ) )

betas <- sapply(regressoes, function(modelo) coef(modelo)[2])

beta1 = mean( betas )

In [ ]:
# Gráfico dos betas
ggplot() +
  geom_histogram( aes(x = betas), bins = 30, color = "black", fill = "darkred", alpha = 0.8) +
  labs(title = "Histograma dos betas estimados", x = NULL, y = "Densidade") +
  theme_classic() +
  theme(
    plot.title = element_text(size = 18, face = "bold"),
    axis.title = element_text(size = 14, face = "bold"),
    axis.text = element_text(size = 12),
    strip.text = element_text(size = 12, face = "bold")
  )

E se aumentarmos o tamanho da amostra para 2000 elementos em cada uma das 500 amostras?

In [ ]:
# Para replicarmos as variáveis pseudo aleatórias
set.seed(13)

# Definindo os parâmetros

amostras <- 500 # número de amostras

n <- 2000        # tamanho de cada amostra

b0 <- 2

b1 <- 3

# Criando as amostras

X <- replicate( amostras, rnorm( n, mean = 10, sd = 2 ) )

e <- replicate( amostras, rnorm( n, mean = 0, sd = 1 ) )

Y <- b0 + b1 * X + e

In [ ]:
# Fazendo as regressões

regressoes <- lapply( 1:amostras,
                      function(i) lm( Y[ , i ] ~ X[ , i ] ) )

betas <- sapply(regressoes, function(modelo) coef(modelo)[2])

beta1 = mean( betas )

In [ ]:
# Gráfico dos betas
ggplot() +
  geom_histogram( aes(x = betas), bins = 30, color = "black", fill = "darkred", alpha = 0.8) +
  labs(title = "Histograma dos betas estimados", x = NULL, y = "Densidade") +
  theme_classic() +
  theme(
    plot.title = element_text(size = 18, face = "bold"),
    axis.title = element_text(size = 14, face = "bold"),
    axis.text = element_text(size = 12),
    strip.text = element_text(size = 12, face = "bold")
  )

Note que a variabilidade diminuiu!

Vamos verificar a consistência do estimador:

In [ ]:
tamanhos = c( seq( from = 5, to = 90, by = 5 ),  seq( from = 100, to = 10000, by = 500 ) )

amostras = length( tamanhos ) # número de amostras

beta_mean = rep( NA, amostras )

for ( n in 1:amostras ) {

  set.seed( 13 + n )

  b0 <- 2

  b1 <- 3

  # Criando as amostras

  X <- replicate( amostras, rnorm( tamanhos[ n ], mean = 10, sd = 2 ) )

  e <- replicate( amostras, rnorm( tamanhos[ n ], mean = 0, sd = 1 ) )

  Y <- b0 + b1 * X + e

  regressoes <- lapply( 1:amostras,
                        function(i) lm( Y[ , i ] ~ X[ , i ] ) )

  betas <- sapply(regressoes, function(modelo) coef(modelo)[2])

  beta_mean[ n ] = mean( betas ) - b1


}

In [ ]:
data = data.frame( beta_mean, tamanhos  )

ggplot(data) +
  geom_line( aes(x = tamanhos, y = beta_mean), colour = "darkred", size = 2) +
  theme_classic() +
  labs(   title = "Erro amostral como função do tamanho da amostra",
              x = "",
              y = "",
        caption = "Diferença entre  estimador e o parâmetro estimado em regressões lineares,
                         Y = b0 + b X, para diferentes ramanhos de amostra" )

                           labs(, x = NULL, y = "Densidade") +
  theme_classic() +
  theme(
    plot.title = element_text(size = 24, face = "bold"),
    axis.title = element_text(size = 14, face = "bold"),
    axis.text = element_text(size = 18),
    strip.text = element_text(size = 14, face = "bold") )